<a href="https://colab.research.google.com/github/Sai3851/Data-driven-Defect-Prediction/blob/main/Copy_of_Copy_of_AEEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/Defect Prediction Project/archive-AEEM.zip' -d '/content/drive/MyDrive/Defect_Prediction_Datasets/'


Archive:  /content/drive/MyDrive/Defect Prediction Project/archive-AEEM.zip
  inflating: /content/drive/MyDrive/Defect_Prediction_Datasets/EQ.csv  
  inflating: /content/drive/MyDrive/Defect_Prediction_Datasets/JDT.csv  
  inflating: /content/drive/MyDrive/Defect_Prediction_Datasets/LC.csv  
  inflating: /content/drive/MyDrive/Defect_Prediction_Datasets/ML.csv  
  inflating: /content/drive/MyDrive/Defect_Prediction_Datasets/PDE.csv  


In [ ]:
import pandas as pd

# Loading each dataset
eq_data = pd.read_csv('/content/drive/MyDrive/Defect_Prediction_Datasets/EQ.csv')
jdt_data = pd.read_csv('/content/drive/MyDrive/Defect_Prediction_Datasets/JDT.csv')
lc_data = pd.read_csv('/content/drive/MyDrive/Defect_Prediction_Datasets/LC.csv')
ml_data = pd.read_csv('/content/drive/MyDrive/Defect_Prediction_Datasets/ML.csv')

# Displaying the first few rows of each dataset to check structure
print("EQ Data:\n", eq_data.head())
print("JDT Data:\n", jdt_data.head())
print("LC Data:\n", lc_data.head())
print("ML Data:\n", ml_data.head())


EQ Data:
    ck_oo_numberOfPrivateMethods  LDHH_lcom  LDHH_fanIn  \
0                             3   0.002547    0.002555   
1                            37   0.008643    0.004756   
2                             3   0.001479    0.009143   
3                            10   0.005642    0.005395   
4                             1   0.001350    0.000000   

   numberOfNonTrivialBugsFoundUntil:  WCHU_numberOfPublicAttributes  \
0                                  4                           0.00   
1                                 71                           0.00   
2                                  5                           1.01   
3                                 38                           0.00   
4                                  1                           1.01   

   WCHU_numberOfAttributes  CvsWEntropy  LDHH_numberOfPublicMethods  \
0                     3.04     0.393707                    0.003049   
1                    14.37     2.093750                    0.001481   
2

In [ ]:
# Applying fillna() only to numeric columns
eq_data.fillna(eq_data.select_dtypes(include=['float64', 'int64']).mean(), inplace=True)
jdt_data.fillna(jdt_data.select_dtypes(include=['float64', 'int64']).mean(), inplace=True)
lc_data.fillna(lc_data.select_dtypes(include=['float64', 'int64']).mean(), inplace=True)
ml_data.fillna(ml_data.select_dtypes(include=['float64', 'int64']).mean(), inplace=True)


In [ ]:
from sklearn.preprocessing import StandardScaler

# Combining EQ and JDT for training
train_data = pd.concat([eq_data, jdt_data], ignore_index=True)
test_data = pd.concat([lc_data, ml_data], ignore_index=True)

# Identifying numeric columns
numeric_columns = train_data.select_dtypes(include=['float64', 'int64']).columns

# Initializing scaler
scaler = StandardScaler()
train_data[numeric_columns] = scaler.fit_transform(train_data[numeric_columns])
test_data[numeric_columns] = scaler.transform(test_data[numeric_columns])  # Use the same scaler for test data


In [ ]:
train_data = pd.get_dummies(train_data, drop_first=True)
test_data = pd.get_dummies(test_data, drop_first=True)

# Ensure both train and test data have the same columns
test_data = test_data.reindex(columns=train_data.columns, fill_value=0)


In [ ]:
X_train = train_data.drop('class_clean', axis=1)  # Replace 'class' with the actual target column
y_train = train_data['class_clean']

X_test = test_data.drop('class_clean', axis=1)
y_test = test_data['class_clean']


In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


In [ ]:
from tensorflow.keras import layers, models

encoder = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_resampled.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),  # Latent space
    layers.Dense(1, activation='sigmoid')  # Output for binary classification
])

encoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
history = encoder.fit(X_train_resampled, y_train_resampled, epochs=20, validation_data=(X_test, y_test))


Epoch 1/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.6120 - loss: 0.6426 - val_accuracy: 0.8018 - val_loss: 0.5358
Epoch 2/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7501 - loss: 0.4947 - val_accuracy: 0.7697 - val_loss: 0.5910
Epoch 3/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8043 - loss: 0.4344 - val_accuracy: 0.7705 - val_loss: 0.6233
Epoch 4/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8325 - loss: 0.3995 - val_accuracy: 0.7403 - val_loss: 0.7278
Epoch 5/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8503 - loss: 0.3646 - val_accuracy: 0.6941 - val_loss: 0.8265
Epoch 6/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8509 - loss: 0.3540 - val_accuracy: 0.7908 - val_loss: 0.8129
Epoch 7/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8735 - loss: 0.3146 - val_accuracy: 0.7556 - val_loss: 0.8752
Epoch 8/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8972 - loss: 0.2639 - val_accuracy: 0.7493 - val_los

In [ ]:
test_loss, test_accuracy = encoder.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")


80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6341 - loss: 1.6663
Test Accuracy: 0.7329


In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

y_pred = (encoder.predict(X_test) > 0.5).astype(int)
print(classification_report(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_pred))


80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
              precision    recall  f1-score   support

       False       0.13      0.22      0.16       309
        True       0.88      0.80      0.84      2244

    accuracy                           0.73      2553
   macro avg       0.51      0.51      0.50      2553
weighted avg       0.79      0.73      0.76      2553

AUC: 0.5103750237959261


In [ ]:
import tensorflow as tf

def rbf_kernel(x, y, gamma=1.0):
    # Compute the RBF kernel between x and y
    xx = tf.expand_dims(x, 1)
    yy = tf.expand_dims(y, 0)
    distances = tf.reduce_sum((xx - yy) ** 2, axis=-1)
    return tf.exp(-gamma * distances)

def compute_mmd_loss(source_features, target_features, gamma=1.0):
    # Calculate MMD loss
    source_kernel = rbf_kernel(source_features, source_features, gamma)
    target_kernel = rbf_kernel(target_features, target_features, gamma)
    cross_kernel = rbf_kernel(source_features, target_features, gamma)

    mmd = tf.reduce_mean(source_kernel) + tf.reduce_mean(target_kernel) - 2 * tf.reduce_mean(cross_kernel)
    return mmd


In [ ]:
# Placeholder for source and target data loading (train and test datasets)
source_data = X_train_resampled  # Training data
target_data = X_test  # Testing data

# Training function with MMD integration
def train_with_mmd(encoder, source_data, y_source, target_data, epochs=20, gamma=1.0):
    optimizer = tf.keras.optimizers.Adam()

    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            # Forward pass
            source_features = encoder(source_data, training=True)
            target_features = encoder(target_data, training=True)

            # Compute binary cross-entropy loss
            predictions = encoder(source_data)
            y_source = tf.reshape(y_source, (-1, 1))
            classification_loss = tf.keras.losses.binary_crossentropy(y_source, predictions)

            # Compute MMD loss
            mmd_loss = compute_mmd_loss(source_features, target_features, gamma=gamma)

            # Combine losses
            total_loss = tf.reduce_mean(classification_loss) + mmd_loss

        # Backpropagation
        gradients = tape.gradient(total_loss, encoder.trainable_variables)
        optimizer.apply_gradients(zip(gradients, encoder.trainable_variables))

        # Print progress
        print(f"Epoch {epoch+1}/{epochs}, Total Loss: {total_loss.numpy()}, MMD Loss: {mmd_loss.numpy()}")

# Run training with MMD
train_with_mmd(encoder, X_train_resampled, y_train_resampled, X_test)


Epoch 1/20, Total Loss: 0.27712583541870117, MMD Loss: 0.14350998401641846
Epoch 2/20, Total Loss: 0.25131887197494507, MMD Loss: 0.14332127571105957
Epoch 3/20, Total Loss: 0.25179973244667053, MMD Loss: 0.13374876976013184
Epoch 4/20, Total Loss: 0.24138525128364563, MMD Loss: 0.12337684631347656
Epoch 5/20, Total Loss: 0.2251759022474289, MMD Loss: 0.11150825023651123
Epoch 6/20, Total Loss: 0.21316802501678467, MMD Loss: 0.09875738620758057
Epoch 7/20, Total Loss: 0.20886625349521637, MMD Loss: 0.08652091026306152
Epoch 8/20, Total Loss: 0.20660610496997833, MMD Loss: 0.077628493309021
Epoch 9/20, Total Loss: 0.19844381511211395, MMD Loss: 0.07114219665527344
Epoch 10/20, Total Loss: 0.18867594003677368, MMD Loss: 0.06667006015777588
Epoch 11/20, Total Loss: 0.1826220452785492, MMD Loss: 0.06353569030761719
Epoch 12/20, Total Loss: 0.17954254150390625, MMD Loss: 0.05992698669433594
Epoch 13/20, Total Loss: 0.17597109079360962, MMD Loss: 0.05478525161743164
Epoch 14/20, Total Loss: 

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

# Make predictions on the test data
y_pred = (encoder.predict(X_test) > 0.5).astype(int)  # Convert predictions to binary values

# Evaluate accuracy and loss on the test set
test_loss, test_accuracy = encoder.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Print detailed classification metrics
print(classification_report(y_test, y_pred))

# Calculate and display AUC
auc_score = roc_auc_score(y_test, y_pred)
print("AUC:", auc_score)


80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5060 - loss: 2.7977
Test Accuracy: 0.6048
              precision    recall  f1-score   support

       False       0.17      0.56      0.26       309
        True       0.91      0.61      0.73      2244

    accuracy                           0.60      2553
   macro avg       0.54      0.59      0.49      2553
weighted avg       0.82      0.60      0.67      2553

AUC: 0.585416558503366
